## Week 5 practice

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/refs/heads/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv")

df.columns = df.columns.str.lower().str.replace(" ", "_")
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')


df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [3]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state=1)

In [4]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
       'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [5]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)
    
    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)

    return dv, model

In [6]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X=dv.transform(dicts)
    y_pred = model.predict_proba(X)[:,1]

    return y_pred

In [7]:
C = 1.0
n_splits = 5

In [8]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
        df_train = df_full_train.iloc[train_idx]
        df_val = df_full_train.iloc[val_idx]
    
        y_train = df_train.churn.values
        y_val = df_val.churn.values
        
        dv, model = train(df_train, y_train, C=C)
        y_pred = predict(df_val, dv, model)
        auc = roc_auc_score(y_val, y_pred)
        scores.append(auc)

print('C=%s %.5f +- %.5f' % (C, np.mean(scores), np.std(scores)))


/Users/AKhvatov/.local/share/virtualenvs/machine-learning-zoomcamp-homework-bLdGrrwV/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/AKhvatov/.local/share/virtualenvs/machine-learning-zoomcamp-homework-bLdGrrwV/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to

C=1.0 0.84192 +- 0.00696


/Users/AKhvatov/.local/share/virtualenvs/machine-learning-zoomcamp-homework-bLdGrrwV/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
scores

[np.float64(0.8446632807655172),
 np.float64(0.8451798602834061),
 np.float64(0.8332610876981738),
 np.float64(0.8347129404617255),
 np.float64(0.8517892441404412)]

In [10]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

y_test = df_test.churn.values

auc = roc_auc_score(y_test, y_pred)
auc

/Users/AKhvatov/.local/share/virtualenvs/machine-learning-zoomcamp-homework-bLdGrrwV/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


np.float64(0.8582840412969764)

### Save the model

In [11]:
import pickle

In [12]:
output_file = f"model_C={C}.bin"
output_file

'model_C=1.0.bin'

In [13]:
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)
    

### Load the model

Here we restarted the kernel

In [14]:
import pickle

input_file = 'model_C=1.0.bin'

In [15]:
with open(input_file, 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [16]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=1000))

In [17]:
customer = {
'gender':'female',
    'seniorcitizen':0,
    'partner':'yes',
    'dependents':'no',
    'phoneservice':'no',
    'multiplelines':'no_phone_service',
    'internetservice':'dsl',
    'onlinesecurity':'no',
    'onlinebackup':'yes',
    'deviceprotection':'no',
    'techsupport':'no',
    'streamingtv':'no',
    'streamingmovies':'no',
    'contract':'month-to-month',
    'paperlessbilling':'yes',
    'paymentmethod':'electronic_check',
    'tenure':1,
    'monthlycharges':29.85,
    'totalcharges':29.85
    
}

In [18]:
X=dv.transform([customer])

In [19]:
model.predict_proba(X)[0,1]

np.float64(0.627562406232732)

In [20]:
#probably want to send her the promo